In [2]:
import time
import numpy as np
from itertools import product
from scipy.spatial import cKDTree
import open3d as o3d

def random_orthogonal_matrix(d=3):
    H = np.random.randn(d, d)
    Q, _ = np.linalg.qr(H)
    return Q

def reflection_group(d=3):
    return [np.diag(signs) for signs in product([-1, 1], repeat=d)]

# Load full point cloud once
pcd = o3d.io.read_point_cloud("/Users/judahlevin/UATX/Linear Algebra/Final Project/Models/horse.ply")
X_full = np.asarray(pcd.points)
n_full = X_full.shape[0]

rotation_errors = []

# Start timing
start_time = time.time()

for seed in range(100):
    np.random.seed(seed)

    # -------------------------------
    # STEP 1: Load and Subsample Data
    # -------------------------------
    n_sample = 1000
    indices = np.random.choice(n_full, n_sample, replace=False)
    X = X_full[indices]
    n = X.shape[0]
    P = X - X.mean(axis=0)

    # -------------------------------
    # STEP 2: Generate Ground-Truth Transformed Cloud Q = O @ S @ P
    # -------------------------------
    O = random_orthogonal_matrix()
    perm = np.random.permutation(n)
    S = np.eye(n)[perm]
    Q_true = (O @ (S @ P).T).T
    Q = Q_true - Q_true.mean(axis=0)

    # -------------------------------
    # STEP 3: Kolpakov's E–Init
    # -------------------------------
    E_P = P.T @ P
    E_Q = Q.T @ Q
    _, U_P = np.linalg.eigh(E_P)
    _, U_Q = np.linalg.eigh(E_Q)
    U0 = U_Q @ U_P.T

    best_U = None
    best_error = float("inf")
    best_indices = None
    best_matched_P = None

    for D in reflection_group():
        U_candidate = U0 @ U_P @ D @ U_P.T
        Q_init_candidate = Q @ U_candidate.T
        _, indices = cKDTree(P).query(Q_init_candidate)
        matched_P = P[indices]
        error = np.linalg.norm(matched_P - Q_init_candidate)
        if error < best_error:
            best_error = error
            best_U = U_candidate
            best_indices = indices
            best_matched_P = matched_P

    # -------------------------------
    # STEP 4: Final Aligned Output (No ICP)
    # -------------------------------
    Q_init = Q @ best_U.T

    # -------------------------------
    # STEP 5: Kolpakov Evaluation Metrics (E–Init only)
    # -------------------------------
    delta = np.linalg.norm(best_matched_P - Q_init)
    delta_o = np.linalg.norm(best_U - O)
    true_sigma = np.argmax(S, axis=0)
    S_true = np.eye(n)[perm]
    S_est = np.eye(n)[best_indices]
    delta_H = np.linalg.norm(S_est - S_true, ord='fro')**2 / (2 * n)

    rotation_errors.append(delta_o)

# End timing
end_time = time.time()
total_time = end_time - start_time
avg_time_per_seed = total_time / 100

# -------------------------------
# Summary of Rotation Error
# -------------------------------
rotation_errors = np.array(rotation_errors)
print("\n=== Kolpakov E–Init-Only Rotation Error Summary over 100 Seeds ===")
print(f"Mean δ_o   (rotation error):       {rotation_errors.mean():.6f}")
print(f"Median δ_o (rotation error):       {np.median(rotation_errors):.6f}")
print(f"Std δ_o    (rotation error):       {rotation_errors.std():.6f}")
print(f"\nAverage time per seed loop:        {avg_time_per_seed:.6f} seconds")


=== Kolpakov E–Init-Only Rotation Error Summary over 100 Seeds ===
Mean δ_o   (rotation error):       1.783305
Median δ_o (rotation error):       2.000000
Std δ_o    (rotation error):       1.208231

Average time per seed loop:        0.042082 seconds
